# Recommendation Systems & Non-negative Matrix Factorization

For this project, we will implement and replicate the techniques used in the paper _Learning from Incomplete Ratings Using Non-negative Matrix Factorization_, as well as other techniques that solve more optimally issues of initialization and learning.

* Firstly, the dataset that we will use is going to be the Movielens 1M; A collection of ratings of movies by corresponding users. Each user may rate a number of movies in a range of 1 to 5 stars. Of course, we have no information regarding the rating that a user could give to the unrated movies.
* Our main objective revolves around completing the _users-to-movies_ matrix in order to create qualitative predictions about the unknown values.

#### We start by importing the necessary frameworks and importing the dataset.

In [4]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
from numpy import linalg as LA
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook, show

In [ ]:
# Import the ratings .dat file.
R = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python')
R.columns = ['user', 'movie', 'rating', 'timestamp']

#### Create the rating matrix & initialize hyperparameters.

The number of unique users and movies will represent the 2 dimensions of our final rating matrix. The _pivot()_ function will create a matrix with the 'user' column as index, and the 'movie' column as the potential columns for each user. Of course this will result in a matrix with lots of unknown values. These values will be filled with zeros, and the correct prediction of these will be the main goal of this project.

In [ ]:
numUsers = len(R['user'].unique())
numMovies = len(R['movie'].unique())

# Create Users x Movies ratings matrix.
Adf = R.pivot(index = 'user', columns = 'movie', values = 'rating').fillna(0)

print('Sparsity percentage : ' + str(1.0 - np.count_nonzero(Adf.values)/float(numUsers * numMovies)) )
print('Number of Users : ' + str(numUsers))
print('Number of Movies : ' + str(numMovies))

#### NMF

The algorithm that we will use for performing NMF on the ratings matrix will be the Multiplicative Updates as shown in the work of _Lee & Seung_. The rules for updating W and H will be performed iteratively, initially computing the new iteration of H with given W, and then computing the new W given the new H. 

$$ H = H \frac{W^T}{W^TWH}$$
$$ W = W \frac{AH^T}{WHH^T}$$

#### NMF Initialization ( Random & NNDSVD )

Regarding the initialization of the matrices of the decomposition W and H, the paper does not propose any methods. According to specific sections in _Learning from Incomplete Ratings Using Non-negative Matrix Factorization_, W and H are randomly initialized. 

In order to perform an initialization that can achieve a faster convergence, we utilize the Non Negative Double Singular Value Decomposition method also called _NNDSVD_.

The steps to perform _NNDSVD_ are as the following :
* Choose a number of factors _k_
* Find the _k_ largest singular triplets of A through SVD
* Initialize first column of _W_ and first row of _H_ from _Perron-Frobenius_ theorem
* For _i_ from row/column to _k_ compute :
    * $U_+[:,i]$ ∘ (A) and $U_-[:,i]$ ∘ (-A)
    * $V_+[i,:]$ ∘ (A) and $V_-[i,:]$ ∘ (-A)
    * Compute the norms of all 4, as well as the product of the positive/negative norm equivalents correspondingly.
    * If the product (_mp_) of the positive-part norms is bigger than the negative-part (_mn_), then the _i-th_ column/row of W/H are $$W[:,i] = \sqrt{mp * \Sigma_i} * \frac{U_+[:,i] ∘ (A)}{norm(U_+[:,i] ∘ (A))}$$ $$H[i,:] = \sqrt{mp * \Sigma_i} * \frac{V_+[i,:] ∘ (A)}{norm(V_+[i,:] ∘ (A))}^T$$
    * Else if (_mn_) > (_mp_) then $$W[:,i] = \sqrt{mn * \Sigma_i} * \frac{U_-[:,i] ∘ (A)}{norm(U_-[:,i] ∘ (A))}$$ $$H[i,:] = \sqrt{mn * \Sigma_i} * \frac{V_-[i,:] ∘ (A)}{norm(V_-[i,:] ∘ (A))}^T$$

#### Expectation-Maximization Procedure 

The algorithm of EM consists of two phases, during which we attempt to learn the maximum likelihood parameters of a model with incomplete data. 

The first phase of the algorithm known as the _Expectation_ step, given the observed data of the known ratings and the previous model estimate X = W * H where (W, H) are the factorization matrices, computes a new estimation for the model X with respect to the unknown data. 

$$ Q(X, X^{(t-1)}) = E[logPr(A^o , A^u | X) | A^o , X^{(t-1)}]$$

The second phase of the EM algorithm known as the _Maximization_ step, takes the current model estimation X of the ongoing _t-th_ iteration and tries to maximize the immediately previous computed expectation. 

$$ X^{(t)} = arg_XmaxQ(X, X^{(t-1)}) $$

Regarding the unknown entries, we know that $$A_{ij} \sim N(X_{ij}, \sigma^2)$$ 
Τhus the expected estimation will be equal to the corresponding parameter of $$A_{ij}^u = X_{ij}^{(t-1)}$$

Therefore A' is a denotation of the matrix A where the observed values remain the same and the unknown entries are completed through the current model estimate X as an Expectation step, and then we update the model parameters in a Maximization step. 

In conclusion, the problem is summarized into minimizing the objective function $$||A' - W*H||_F^2$$ which can be resolved by performing NMF on A'. 

In [ ]:
def EM(Adf, W, H, iterations, rounds):
    
    ### Expectation-Maximization (EM) Procedure
    A = Adf.values
    maeEM = [] 
    
    # 1st MAE is from NNDSVD raw
    X = np.dot(W,H)
    maeEM.append(sum(sum(abs(A-X))/(numUsers*numMovies)))
    
    # Estimation Phase : Complete matrix A with current estimation of W*H
    zeroIndexX = np.where(A == 0)[0]
    zeroIndexY = np.where(A == 0)[1]
    
    for ii in range(iterations): 
    
        X = np.dot(W,H)
        for ind in range(0,len(zeroIndexX)):
            A[ zeroIndexX[ind] , zeroIndexY[ind] ] = X[ zeroIndexX[ind] , zeroIndexY[ind] ]
           
        for i in range(rounds):    
            
            # Maximization Phase : Train the factorization matrices W, H to minimize the objective function
            H = H * ( np.dot( np.transpose(W), A ) ) / ( np.dot( np.transpose(W) , np.dot(W,H) ) )
            W = W * ( np.dot( A, np.transpose(H)) ) / ( np.dot( W , np.dot(H, np.transpose(H) ) ) )
            
            # Append the new and smaller objective function MAE
            X = np.dot(W,H)
            maeEM.append(sum(sum(abs(A-X))/(numUsers*numMovies)))
    
    return maeEM

#### Weighted Non-negative Matrix Factorization Procedure 

The novel idea that is introduced in the _WNMF_ method, is a new matrix _B_ that contains either zeros and ones. The element $B_{ij}$ is either 1 if the value $ \in A^o$ which means that it is an observed rating given by the rating dataset, or 0 if the value $ \in A^u$ thus denoting that the value is not zero rather that it is unknown or unobserved and must be computed.

The new updating formulas now become :

$$W = W \frac{(B*A)H^T}{(B*(WH))H^T} $$
$$H = H \frac{W^T(B*A)}{W^T(B*(WH))} $$

#### Handling Zeros in the Denominator

During the EM procedure, the Expectation step made sure that the ratings matrix A would fill the unobserved or unknown values with the current estimate X. Thus, during the multiplicative updates, there were no division by zero during the component-wise division of the Lee&Seung algorithm, therefore no issue had emerged. 

In the WNMF procedure, this problem persists. Because neither matrix A is completed with estimated values, nor matrix B has many non-zero values, so the multiplicative updates will bring us to many divisions by zero. The problem is handled through a small variable named _delta_, which we add to the denominator. Of course, this _delta_ is a hyperparameter whose behaviour must be observed through the results by testing various values to assign to it. 

The updating formulas with respect to the zeros in the denominator :

$$W = W \frac{(B*A)H^T}{(B*(WH) + \delta)H^T} $$
$$H = H \frac{W^T(B*A)}{W^T(B*(WH) + \delta)} $$

In [ ]:
def WNMF(Adf, W, H, delta, rounds):

    ### Weighted NMF
    A = Adf.values
    maeWNMF = [] 
    
    # 1st MAE is from NNDSVD raw
    X = np.dot(W,H)
    maeWNMF.append(sum(sum(abs(A-X))/(numUsers*numMovies)))
    
    # Create weights matrix. 
    zeroIndexX = np.where(A == 0)[0]
    zeroIndexY = np.where(A == 0)[1]
    B = np.ones((numUsers, numMovies))
    
    for ind in range(0,len(zeroIndexX)):
        B[ zeroIndexX[ind] , zeroIndexY[ind] ] = 0
        
    for i in range(rounds):    
        
        # Train the factorization matrices W, H to minimize the objective function
        H = H * ( np.dot( np.transpose(W), (B*A) ) ) / ( np.dot( np.transpose(W) , (B*np.dot(W,H) + delta) ) )
        W = W * ( np.dot( (B*A), np.transpose(H)) ) / ( np.dot( (B*np.dot(W,H) + delta) , np.transpose(H) ) )
        
        # Append the new and smaller objective function MAE
        X = np.dot(W,H)
        maeWNMF.append(sum(sum(abs(A-X))/(numUsers*numMovies)))
    
    return maeWNMF

#### Projected Barzilai-Borwein Method

A generalized projected method would take the unput $x_i$ and project it correspondingly, with respect to a predetermined upper and lower bound. 

$$    P[x_i]=\left\{
                \begin{array}{ll}
                  x_i, if l_i < x_i < u_i\\
                  u_i, if x_i \geq u_i\\
                  l_i, if x_i \leq l_i
                \end{array}
              \right. $$
              
$$ d^k = -\nabla f(x^k) $$   

Barzilai-Borwein proposed :

$$ d^k = -\alpha_k\nabla f(x^k) $$

Where $\alpha_k$ :

$$ \alpha_k = \frac{s^{k^T} s^k}{s^{k^T} y^k} $$

$$s^k = x^k - x^{k-1} $$
$$y^k = \nabla (x^k) - \nabla f(x^{k-1})$$

And :

$$    \nabla^Pf(x)_i=\left\{
                \begin{array}{ll}
                  \nabla f(x)_i, if l_i < x_i < u_i\\
                  max(0, \nabla f(x)_i), if x_i \geq u_i\\
                  min(0, \nabla f(x)_i), if x_i \leq l_i
                \end{array}
              \right. $$

The Barzilai-Borwein methods were implemented for unconstrained problems, thus we restrict $\alpha_k$ between two prespecified values $\alpha_{min}$ and $\alpha_{max}$.

The authors also provide insight regarding the hyperparameters that can be optimized and tuned accordingly : -- The selection of $\alpha_k$ -- The direction $d^k$ -- The step size $\lambda_k$.

According to the authors' experiments, $\lambda_k$ does not need to be tuned; it is set equal to 1 as in most cases it is the optimal, thus rendering a few of the algorithm's next steps obsolete. 

Regarding the projected gradient, if the new projected gradient has increased since the last iteration, it could be an omen of instability, and thus we should take a more conservative-sized step, while a big difference through iterations could imply the need for a longer direction of $d^k$ and an according $\alpha_k$.


In [ ]:
def barzilaiStep(A, u, v, aw, rounds, fH_t):
        
    sigma = 0.01
    beta = 0.1
    L = 10
    amin = np.power(10, -9)
    amax = np.power(10, 9)
    
    fbest = fH_t
    grad_fH = np.dot( np.dot( np.transpose(u) , u ) , v ) - np.dot( np.transpose(u), A )
    
    ### step b
    tempD = (v - aw*grad_fH)
    tempD[tempD < 0] = 0
    tempD[tempD > 10000] = 5
    d = tempD - v

    
    ### step c
    g = sum(sum(grad_fH * d))
    y = np.dot( np.dot( np.transpose(u), u ) , d )
    h = sum(sum(y*d))
    
    ### step d
    lamda = 1 
    #print('Shapes are : ' + str(fH_t.shape) + ' + ' + str((lamda*g).shape) + ' + ' + str((0.5*np.power(lamda,2)*h).shape) )
    fH_t_1 = fH_t + lamda*g + 0.5*np.power(lamda,2)*h
    
    #fr = 1000
    #while (fH_t_1 > (fr + sigma*g)):
    #    lamda = lamda*beta
    #    fH_t_1 = fH_t + lamda*g + 0.5*np.power(lamda,2)*h
    
    ### step e
    d = lamda*d
    y = lamda*y
    h = np.power(lamda, 2)*h
    v = v + d
    grad_fH_t_1 = grad_fH + y
    
    ### step f    
    if ( LA.norm(grad_fH_t_1) > LA.norm(grad_fH) ):
        aw = np.amin([np.amax([np.amax(h/sum(sum(y*y))), amin]) , amax])
    else:
        aw = np.amin([np.amax([np.amax(sum(sum((d*d))/h)), amin]) , amax])
    
    ### step g
    stepG = 0
    if (stepG == 1):
        if (fH_t_1 < fbest):
            
            fbest = fH_t_1
            fc = fH_t_1
            l = 0
        
        else:
            
            fc = np.amax([fc,fH_t_1])
            l = l + 1
            
            if(l==L):
                fr = fc
                fc = fH_t_1
                l = 0
    
    return v, aw, fH_t_1

### A few quick comments on issues of the paper

* The paper does not specify which version of the Movielens dataset collection it is using. There exist versions of 1 million ratings, 20 million ratings or even smaller datasets of 100000 ratings. 
* All these have different percentages of sparsity; even if we take different 1500-by-3000 snapshots of the ratings matrices, the behaviour of the algorithms will be different. 
* Thus, we do not have a baseline model of accuracy regarding the final results. 

* Also, the paper does not specify anything regarding the initialization of the matrices _W_ and _H_ which pertain the initial decomposition of _A_ (the ratings matrix). It is implied in later sections that the initializations are random, although without mentioning the range of the values of the randomness.

* Furthermore, the paper does not address at all, the problem of the zero values in the denominators which obviously persists in the _WNMF_ method. ( A proposed solution is presented by Piper et al. in _Object Characterization from Spectral Data Using Nonnegative Factorization and Information Theory_ , where a small number is added to the denominator).

### Results

In [5]:
iterations = 5              # How many EM Iterations ? ( 50 iterations * emRounds = 2000 )
emRounds = 20               # EM rounds per iteration ( 2000/50 = 40 )
rounds = 100                # Rounds of updating WNMF ( 2000 )
delta = 0.1                 # Hyperparameter to manage division by zero 
factors = 20                # Latent factors indicted = 20 by the paper

In [ ]:
# Initializer : NND-SVD    --    Method : E.M.
W = np.zeros((numUsers, factors))
H = np.zeros((factors, numMovies))
W, H = NNDSVD(Adf, factors, W, H)
NNDSVDmaeEM = EM(Adf, W, H, iterations, emRounds)

In [6]:
output_notebook()

Loading BokehJS ...

In [7]:
h1 = np.load('NNDSVDmaeHYBRID1.npy')
h2 = np.load('NNDSVDmaeHYBRID3.npy')
h3 = np.load('NNDSVDmaeHYBRID5.npy')

em1 = np.load('NNDSVDmaeEM.npy')
wnmf1 = np.load('NNDSVDmaeWNMF.npy')
wnmf2 = np.load('NNDSVDmaeWNMF001.npy')
bb1 = np.load('NNDSVDmaeBB.npy')

In [8]:
ticc = [i for i in range(rounds)]                                               # 0, 1, 2, 3, 4, ... , rounds
iterationChangeX = [i for i in range(0,rounds+1,emRounds)]                      # 0, iteration 1 (25), iteration 2 (50), ...
iterationChangeY = [em1[i] for i in range(0,rounds,emRounds)]                   # Above index for the y axis (MAE).

source = ColumnDataSource(data=dict(
    x = ticc[2:],
    y = bb1[1:],
    desc=[bb1[0], bb1[1], bb1[2], bb1[3], bb1[4], bb1[5]],
))

source2 = ColumnDataSource(data=dict(
    x = iterationChangeX[1:],
    y = iterationChangeY[1:],
    desc=[em1[20], em1[40], em1[60], em1[80]],
))


TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("desc", "@desc"),
]

p = figure(
   tools="pan,box_zoom,reset,save",
   y_axis_type="log", title="Baseline Methods Comparison",
   x_axis_label='Rounds', y_axis_label='Mean Absolute Error', 
   tooltips=TOOLTIPS
)

p.xaxis.major_tick_line_color = "firebrick"
p.xaxis.major_tick_line_width = 3
p.xaxis.minor_tick_line_color = "orange"

p.xaxis.ticker = iterationChangeX

p.circle('x', 'y', size=15, source=source2, hover_fill_color="indigo", fill_alpha=0.05, line_color=None, hover_line_color="white")
p.circle('x', 'y', size=15, source=source, hover_fill_color="indigo", fill_alpha=0.05, line_color=None, hover_line_color="white")

p.line(ticc[2:], em1[3:], legend="EM", line_width=2, line_join="bevel", color='salmon')
p.line(ticc[2:], wnmf1[3:], legend="WNMF - delta= 0.1", line_width=2, line_join="bevel", color='indigo')
p.line(ticc[2:], wnmf2[3:], legend="WNMF - delta= 0.01", line_width=2, line_join="bevel", color='orange')
p.line(ticc[2:], bb1[1:], legend="Barzilai-Borwein", line_width=2, line_join="bevel", color='firebrick')

p.legend.location = "top_right"

show(p)

In [9]:
### Visualization

ticc = [i for i in range(rounds)]                                               # 0, 1, 2, 3, 4, ... , rounds
iterationChangeX = [i for i in range(0,rounds+1,emRounds)]                      # 0, iteration 1 (25), iteration 2 (50), ...
iterationChangeY = [h1[i] for i in range(0,rounds,emRounds)]                    # Above index for the y axis (MAE).

p = figure(
   tools="pan,box_zoom,reset,save",
   y_axis_type="log", title="Hybrid Model Comparison",
   x_axis_label='Rounds', y_axis_label='Mean Absolute Error',
)

p.line(ticc[3:], h1[4:], legend="Hybrid -- 1 EM + WNMF", line_width=2, line_join="bevel", color='orange')
p.line(ticc[3:], h2[4:], legend="Hybrid -- 3 EM + WNMF", line_width=2, line_join="bevel", color='salmon')
p.line(ticc[3:], h3[4:], legend="Hybrid -- 5 EM + WNMF", line_width=2, line_join="bevel", color='indigo')

p.legend.location = "center_right"

show(p)

#### Results and Future Work

Regarding the first figure, we see the comparison of the two baseline methods _EM_ and _WNMF_ as well as the implemented method of Barzilai-Borwein. 

The Expectation-Maximization algorithm has a behaviour similar behaviour to the one of the figures in the paper. After running a few inner iterations, the algorithm makes a few steps towards convergence, however the outer iterations of the algorithm where the unknown values are completed with the estimation provide much bigger steps of improvement before it relatively stagnates in a slow manner. 

For the _WNMF_ method, we had to make a decision in regards to the value of the delta. Insighfully, values closer to zero perform actually better, as also seen in the figure. Also, the insights made by the authors of the paper regarding the performance depending a lot on the initial values of the factorization matrices are reflected on the results' difficulty of convergence.

Hybrid methods, pertain the implementation of an EM procedure with one, three, and five iterations as indicted by the paper, followed by a proper _WNMF_ method. The initialization of the W, H matrices by EM, helps the whole procedure substantially, deeming the _5 EM + WNMF_ as the best performing method, as also shown by the paper.

For the final method of Barzilai Borwein,the main idea revolves around projecting the results of the product of the iteration method within the range of the rating, while also implementing a different step of learning rate than the classic Lee-Seung algorithm. While initially resulting in terrible _MAE_ , it is noteworthy that in only 5 iterations it surpasses all other techniques. 

However, its behaviour in the next hundred of iterations slows down significantly which opens a new discussion about hybrid methods. The first obvious idea of mixing EM and BB (_Barzilai-Borwein_) with EM preceding BB with a few iterations, falls immediately short for a number of reasons. Firstly, BB can really quickly fix its problem of W, H within a really small number of iterations, thus it does not need EM. Also, the two methods pertain different learning rates during their attempts of minimizing the objective function ( gradient descent ). Therefore, a potential mix could be the addition of an Expectation-Maximization step inside the existing BB algorithm, without the multiplicative updates but including the $\alpha_k$ of the BB method.